In [ ]:
!pip install openai-whisper
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.0 MB/s 

In [ ]:
import os
import whisper
import warnings
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU")

In [ ]:
#Loading the openai's whisper model for language identification and speech to text conversion

whisper_model = whisper.load_model("base")

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
#Defining function for sentiment analysis
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = hf_model(inputs)
    predictions = tf.nn.softmax(outputs.logits, axis=1)
    label_index = tf.argmax(predictions, axis=1).numpy()[0]
    label = hf_model.config.id2label[label_index]
    return label

In [ ]:
audio_path = input("Enter the audio path : ")

if not os.path.exists(audio_path):
    print("File not exists")

else:

    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    log_mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)
    _, language_probs = whisper_model.detect_language(log_mel)
    language_code = max(language_probs, key=language_probs.get)

    language_map = {
        "en": "English", "hi": "Hindi", "fr": "French", "de": "German",
        "es": "Spanish", "it": "Italian", "zh": "Chinese", "ar": "Arabic",
        "ru": "Russian", "pt": "Portuguese", "ja": "Japanese"
    }
    language_name = language_map.get(language_code, language_code)

    print(f"\nDetected Language: {language_name} ({language_code})")

    result = whisper_model.transcribe(audio_path)
    transcribed_text = result["text"].strip()

    print("Transcribed Text")
    print(transcribed_text)

    if language_code == "en":
        sentiment = analyze_sentiment(transcribed_text)
        print(f" The Sentiment for the given audio is : {sentiment}")
    else:
        print("\nSentiment analysis is only available for English audio.")


Enter the audio path : /content/GET UP AND GRIND - Motivational Speech.mp3

Detected Language: English (en)
Transcribed Text
If you stay in bed, if you stay on the couch, if you stay in your comfort zone, if you only do what is easy, your life will be hard. But if you do what is hard, if you get up, if you grind, if you are relentless, if you work as hard as possible, when other people are slacking off, your life will be easy.
 The Sentiment for the given audio is : POSITIVE


In [ ]:
language_probs['en']

0.9786800146102905